# CIFO Project

In [1]:
import sys
sys.path.append('..')
from copy import deepcopy
import random
from random import sample, shuffle
from statistics import stdev
import pandas as pd

In [2]:
#import players from csv

players = pd.read_csv('../data/players.csv')
print(players)

    Unnamed: 0              Name Position  Skill  Salary (€M)
0            0       Alex Carter       GK     85           90
1            1      Jordan Smith       GK     88          100
2            2     Ryan Mitchell       GK     83           85
3            3    Chris Thompson       GK     80           80
4            4   Blake Henderson       GK     87           95
5            5     Daniel Foster      DEF     90          110
6            6     Lucas Bennett      DEF     85           90
7            7       Owen Parker      DEF     88          100
8            8      Ethan Howard      DEF     80           70
9            9        Mason Reed      DEF     82           75
10          10      Logan Brooks      DEF     86           95
11          11      Caleb Fisher      DEF     84           85
12          12     Nathan Wright      MID     92          120
13          13      Connor Hayes      MID     89          105
14          14      Dylan Morgan      MID     91          115
15      

In [8]:
players.shape

(35, 5)

In [3]:
names = players['Name'].tolist()
positions = players['Position'].tolist()
skills = players['Skill'].tolist()
salaries = players['Salary (€M)'].tolist()

capacity = 750
num_teams=5
team_size=7

## Sports League Optimization Solution

In [21]:
#Creating the Solution for Sports League Optimization (SLO)
from library.solution import Solution
"""Problem definition
Problem: Sports League Optimization (SLO)
Search space: All possible assignments of 35 players into 5 teams, each with 7 players (non-overlapping). Each team must be composed of: 1GK, 2DEF, 2MID, 2FWD.
Representation: List of 5 teams each one with a list of 7 players.
Fitness function: f(x)= standard deviation of the average skill rating of the teams.
Neighborhood: Swap two players with the same position between two teams.
Goal: Minimize f(x).
"""

class SLOSolution(Solution):

    def __init__(
            self,
            names: list[str]=names,
            positions : list[str]=positions,
            skills : list[int]=skills,
            salaries : list[int]=salaries,
            capacity: int = capacity,
            repr=None
    ):
        self.names = names
        self.positions = positions
        self.skills = skills
        self.salaries = salaries
        self.num_teams = num_teams
        self.team_size = team_size
        self.capacity = capacity

        # Grouping players by position
        self.players_by_position = {
            'GK': [i for i, position in enumerate(self.positions) if position == 'GK'],
            'DEF': [i for i, position in enumerate(self.positions) if position == 'DEF'],
            'MID': [i for i, position in enumerate(self.positions) if position == 'MID'],
            'FWD': [i for i, position in enumerate(self.positions) if position == 'FWD'],
        }


        if repr:
            repr = self._validate_repr(repr)

        super().__init__(repr = repr)


    #MAYBE A BETER VALIDATE FUNCTION IS NEEDED
    #--------------------------------------------------------
    def _validate_repr(self,repr):
        # Check if the representation is a list of teams
        if not isinstance(repr, list):
            raise ValueError("Representation must be a list of teams")
        #check if the number of teams is correct
        if len(repr) != self.num_teams:
            raise ValueError("Number of teams must be equal to num_teams")
        #check if each team has the correct number of players
        for team in repr:
            if not isinstance(team, list):
                raise ValueError("Each team must be a list of players")
            if len(team) != self.team_size:
                raise ValueError("Each team must have 7 players")
            

        #check if team is valid (1GK, 2DEF, 2 MID, 2FWD)
        seen_players = set()

        for i, team in enumerate(repr):
            team_positions = {'GK': 0, 'DEF': 0, 'MID': 0, 'FWD': 0}
            team_salary = 0

            seen_players = set()
            for player_idx in team:
                # Check that the player hasn't already been assigned
                if player_idx in seen_players:
                    raise ValueError(f"Player {player_idx} assigned to multiple teams")
                seen_players.add(player_idx)

                pos = self.positions[player_idx]
                sal = self.salaries[player_idx] # TO BE CONFIRMED

                if pos not in team_positions:
                    raise ValueError(f"Invalid position; '{pos}', for player {player_idx}")
                team_positions[pos] += 1 
                team_salary += sal # TO BE CONFIRMED

            
            if team_positions != {'GK': 1, 'DEF': 2, 'MID': 2, 'FWD': 2}:
                raise ValueError(f"Team {i} has invalid composition: {team_positions}")
            
            # TO BE CONFIRMED
            if team_salary > self.capacity:
                raise ValueError(f"Team {i} exceeds salary cap: {team_salary:.2f}M > {self.capacity}M")
        return repr
        
    
    def random_initial_representation(self):
        isValid = False

        while not isValid:
            try:
                repr = []
                used_players = set()

                for _ in range(self.num_teams):
                    team = []
                    gk = random.choice([p for p in self.players_by_position['GK'] if p not in used_players])
                    used_players.add(gk)
                    team.append(gk)

                    def_players = random.sample([p for p in self.players_by_position['DEF'] if p not in used_players], 2)
                    used_players.update(def_players)
                    team.extend(def_players)

                    mid_players = random.sample([p for p in self.players_by_position['MID'] if p not in used_players], 2)
                    used_players.update(mid_players)
                    team.extend(mid_players)

                    fwd_players = random.sample([p for p in self.players_by_position['FWD'] if p not in used_players], 2)
                    used_players.update(fwd_players)
                    team.extend(fwd_players)

                    repr.append(team)

                repr = self._validate_repr(repr)
                isValid = True

            except (ValueError, IndexError) as e:
                print(f"Invalid representation: {e}")
                isValid = False

        return repr

                    
    def fitness(self):

        team_avg_skill=[]

        for team in self.repr:
            team_salary = sum(self.salaries[player_idx] for player_idx in team)
            if team_salary > self.capacity:
                return 999

            team_skills=[self.skills[player_idx] for player_idx in team]
            avg_skill=sum(team_skills)/len(team_skills)
            team_avg_skill.append(avg_skill)
        
        return stdev(team_avg_skill)

In [20]:
#create a team to test the code
slo_solution = SLOSolution(
    names=names, 
    positions=positions, 
    skills=skills, 
    salaries=salaries, 
    capacity=capacity, 
    repr=None
)

random_teams = slo_solution.random_initial_representation()
print("Random teams:") 
for i, team in enumerate(random_teams):
    print(f"Team {i+1}: {[names[player_idx] for player_idx in team]}")
    #check fitness of the repr
print(f"Fitness: {slo_solution.fitness()}") #TO BE CONFIRMED

Random teams:
Team 1: ['Ryan Mitchell', 'Mason Reed', 'Logan Brooks', 'Bentley Rivera', 'Gavin Richardson', 'Zachary Nelson', 'Elijah Sanders']
Team 2: ['Jordan Smith', 'Caleb Fisher', 'Ethan Howard', 'Spencer Ward', 'Austin Torres', 'Julian Scott', 'Sebastian Perry']
Team 3: ['Alex Carter', 'Daniel Foster', 'Brayden Hughes', 'Dylan Morgan', 'Dominic Bell', 'Tyler Jenkins', 'Xavier Bryant']
Team 4: ['Blake Henderson', 'Jaxon Griffin', 'Maxwell Flores', 'Nathan Wright', 'Hunter Cooper', 'Landon Powell', 'Chase Murphy']
Team 5: ['Chris Thompson', 'Owen Parker', 'Lucas Bennett', 'Connor Hayes', 'Ashton Phillips', 'Adrian Collins', 'Colton Gray']
Fitness: 999


## Mutation Functions

In [ ]:
def mutation1():
    pass

## Crossover Functions

In [ ]:
def crossover1():
    pass

## Selection Functions

In [ ]:
def tournament_selection(population, k=3):

    # Select a random subset of the population for the tournament
    tournament_solutions = random.sample(population, k)

    best = min(tournament_solutions, key=lambda ind: ind.fitness())

    return deepcopy(best)

In [ ]:
def 

## SLO Genetic Algorithm Solution

In [ ]:
class SLOGASolution(SLOSolution):
    def __init__(
            self,
            names,
            positions,
            skills,
            salaries,
            capacity,
            mutation_function,
            crossover_function,
            repr=None,
    ):
        super().__init__(
            names=names,
            positions=positions,
            skills=skills,
            salaries=salaries,
            capacity=capacity,
            repr=repr,
        )
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function
        